In [ ]:
from genparse.lm import TokenGPT2
from transformers import AutoTokenizer, AutoModelForCausalLM
from genparse import CFGLM
from genparse.steer import LocalProduct
from arsenal.maths import softmax, sample
from collections import Counter

In [ ]:
class TokenizingLM:

    def __init__(self, tokenizer, model):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, x):
        return self.model(self.tokenizer.encode(x))

    def p_next(self, x):
        # warning: misalignment is might occur here!
        p = self.model.p_next(self.tokenizer.encode(x))

        assert len(p) == self.tokenizer.vocab_size
        s = [self.tokenizer.decode([k]) for k in range(len(p))]

        # XXX: apparently, multiple tokens in the mapping (token -> substring)
        # can point to the same substring!
        P = Counter()
        for k in range(len(p)):
            P[s[k]] += p[k].item()

        return P


In [ ]:
#gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")

Note that the tokenizer might have multiple tokens that map to the same string.

In [ ]:
#Counter(tokenizer.decode([k]) for k in range(tokenizer.vocab_size)).most_common(20)

In [ ]:
lm = TokenGPT2(gpt_model)

In [ ]:
lm([0,1,2])

In [ ]:
lm.p_next([0,1,2])

In [ ]:
lm.p_next([0,1,2,3])

In [ ]:
p = TokenizingLM(tokenizer, lm).p_next('Once upon a time,')

In [ ]:
p = TokenizingLM(tokenizer, lm).p_next('The following is some code that implements quick sort in Python:')

In [ ]:
pd.DataFrame(p.items()).set_index(0).sort_values(1, ascending=False).head(10)

In [ ]:
M = TokenizingLM(tokenizer, lm)
#.p_terminal('The following is some code that implements quick sort in Python:')

In [ ]:
class Particle:
    def __init__(self, xs):
        self.xs = xs
    def __eq__(self, other):
        return isinstance(other, Particle) and self.xs == other.xs
    def __hash__(self):
        return hash(self.xs)
    def __repr__(self):
        return f'{self.xs}'
    def p(self):
        P = M.p_next(self.xs)
        return Particles({Particle(self.xs + x): w for x, w in P.items()})        

In [ ]:
class Particles(dict):
    def __repr__(self):
        return repr(Counter(self).most_common(10))
    def sample(self):
        ks, ws = np.array(list(self.keys())), np.array(list(self.values()))        
        #ws[np.argsort(-ws)[50:]] = 0
#        return ks[sample(softmax(ws))]
        return ks[sample(ws)]
    def greedy(self):
        ks, ws = list(self.keys()), np.array(list(self.values()))
        return ks[ws.argmax()]

In [ ]:
Particle('The following is some code that implements the quick sort algorithm in Python:')\
.p().sample().p().sample().p().sample().p().sample().p().sample().p().sample().p().sample()\
.p().sample().p().sample().p().sample().p().sample().p().sample().p().sample().p().sample()

In [ ]:
Particle('Once upon a time')\
.p().sample().p().sample().p().sample().p().sample().p().sample().p().sample().p().sample()\
.p().sample().p().sample().p().sample().p().sample().p().sample().p().sample().p().sample()

In [ ]:
p = Particle('Once upon a time,')

In [ ]:
p.p()

In [ ]:
p.p().sample()

In [ ]:
(p.p().greedy().p().greedy().p().greedy().p().greedy().p().greedy().p().greedy()
 .p().greedy().p().greedy().p().greedy().p().greedy().p().greedy().p().greedy()
# .p().greedy().p().greedy().p().greedy().p().greedy().p().greedy().p().greedy()
# .p().greedy().p().greedy().p().greedy().p().greedy().p().greedy().p().greedy()
)